In [2]:
!apt-get install -y poppler-utils
!pip install pdfplumber pytesseract pdf2image openpyxl


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Ign:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5
Err:1 http://security.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5
  404  Not Found [IP: 91.189.91.82 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/main/p/poppler/poppler-utils_22.02.0-2ubuntu0.5_amd64.deb  404  Not Found [IP: 91.189.91.82 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pdfplumber
import pandas as pd
import pytesseract
from pdf2image import convert_from_path
import openpyxl

def extract_tables_from_pdf(pdf_path, page_number):
    tables_data = []
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_number - 1]
        tables = page.extract_tables()

        if not tables:
            print(f"No structured tables found on page {page_number}. Using OCR...")
            text = page.extract_text()
            extracted_data = [[line.strip()] for line in text.split("\n") if line.strip()]
            tables_data.append(pd.DataFrame(extracted_data))
        else:
            for table in tables:
                df = pd.DataFrame(table)

                df = df.applymap(lambda x: "\n".join(line.strip() for line in str(x).splitlines()) if isinstance(x, str) else x)

                tables_data.append(df)
    return tables_data

def perform_ocr_on_page(pdf_path, page_number):
    images = convert_from_path(pdf_path, first_page=page_number, last_page=page_number)
    extracted_text = []
    for image in images:
        text = pytesseract.image_to_string(image)
        extracted_text.append(text)
    return "\n".join(extracted_text)

pdf_files = {
    "cardio_structured.pdf": 6,
    "prot_sap_102.pdf": 50,
    "prot_sap_1.pdf": 14
}

output_file = "extracted_tables.xlsx"
writer = pd.ExcelWriter(output_file, engine='openpyxl')

def adjust_column_width(sheet, dataframe):
    for idx, col in enumerate(dataframe.columns):
        max_length = max([len(str(val).strip()) if val else 0 for val in dataframe[col]])
        adjusted_width = min(max_length + 2, 30)
        sheet.column_dimensions[openpyxl.utils.get_column_letter(idx + 1)].width = adjusted_width

for pdf_name, page_number in pdf_files.items():
    print(f"Processing {pdf_name} - Page {page_number}...")
    pdf_path = f"./{pdf_name}"

    tables = extract_tables_from_pdf(pdf_path, page_number)

    if tables:
        for i, table in enumerate(tables):
            sheet_name = f"{pdf_name}_Page{page_number}_Table{i+1}"[:31]

            table_df = pd.DataFrame(table)

            if pdf_name == "cardio_structured.pdf":
                table_df = table_df.dropna(how="all", axis=1)
                table_df = table_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

            table_df.to_excel(writer, sheet_name=sheet_name, index=False)

            sheet = writer.sheets[sheet_name]
            adjust_column_width(sheet, table_df)

            for row in sheet.iter_rows():
                for cell in row:
                    cell.alignment = openpyxl.styles.Alignment(wrap_text=True, vertical="top")
    else:
        extracted_text = perform_ocr_on_page(pdf_path, page_number)
        df_text = pd.DataFrame([[line.strip()] for line in extracted_text.split("\n") if line.strip()])

        sheet_name = f"{pdf_name}_Page{page_number}_OCR"[:31]
        df_text.to_excel(writer, sheet_name=sheet_name, index=False)

writer.close()
print(f"Extraction completed! Saved as {output_file}")


Processing cardio_structured.pdf - Page 6...


<ipython-input-3-528dba6d2c3d>:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: "\n".join(line.strip() for line in str(x).splitlines()) if isinstance(x, str) else x)
<ipython-input-3-528dba6d2c3d>:64: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  table_df = table_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


Processing prot_sap_102.pdf - Page 50...


<ipython-input-3-528dba6d2c3d>:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: "\n".join(line.strip() for line in str(x).splitlines()) if isinstance(x, str) else x)


Processing prot_sap_1.pdf - Page 14...
Extraction completed! Saved as extracted_tables.xlsx


In [4]:
from google.colab import files
files.download(output_file)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>